# #52 Simple I girder model in shell elements
<i>Steel I girder modelled in shell elements with linear static analysis</i>
***

![Preview](../_img/jupyter_notebook_052.png)

Define a list of segment lengths so the girder can have changes in sections

In [1]:
''' Inputs (Units are N,mm)'''
segments_lengths = [6000, 5000, 6000]  # Each segment of the girder has a different section, defined below
mesh_size = 500

Create a girder class to hold the section dimensions for each segment
We'll create points in the LUSAS model for each section and then use the points to create surfaces joining them.

In [2]:
class GirderSection:
    def __init__(self, tfb:float, tfthk:float, dw:float, tw:float, bfb:float, bfthk:float):
        self.top_flange_breadth = tfb
        self.top_flange_thk = tfthk
        self.web_depth = dw
        self.web_thk = tw
        self.bottom_flange_breadth = bfb
        self.bottom_flange_thk = bfthk
        # We'll save the points created in the model in these lists so we can later use them to define surfaces
        self.bf_points = []
        self.tf_points = []


Create a list of section definitions, we'll need one per segment + 1 for the end.

In [3]:
sections :list[GirderSection] = []
sections.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections.append(GirderSection(500, 50, 1500, 10, 600, 60))
sections.append(GirderSection(500, 30, 1500, 15, 600, 40))
sections.append(GirderSection(500, 30, 1500, 15, 600, 40))
# Check we have defined one more section then segment lengths
assert(len(segments_lengths) == len(sections)-1)

Create a list of stiffener thicknesses that will be positioned at each change in section. A zero thickness indicates no stiffener.

In [4]:
stiffener_thicknesses = [20, 10, 10, 20]

# Ensure number of stiffener thicknesses matches the number of segments
assert(len(sections) == len(stiffener_thicknesses))

### Connect to LUSAS Modeller and create the model

In [5]:
import sys; sys.path.append('../') # Reference modules in parent directory
from shared.LPI import *
lusas = get_lusas_modeller()
if lusas.existsDatabase() and lusas.db().isModified():
    raise Exception("This script will create a new model. Please save or close the current model and try again")

# Create a new model
lusas.newProject("Structural", "Simple_I_Girder_Shell_Model.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("N,mm,t,s,C")

Create a helper function to create a point in the model. The db.createPoint() function returns an IFObjectSet which can contain multiple objects. Since we have only created a single point we'll get a reference to the point and return it

In [6]:
def create_point_in_modeller(x:float, y:float, z:float) -> 'IFPoint':
    return db.createPoint(x, y, z).getObject("Point")

Create all the points for each section, saving the returned point in the Girder section definition

In [7]:
s:GirderSection
x = 0
for i in range(0, len(sections)):
    # Determine the longitudinal position (x coordinate) for each section
    if i > 0 : x += segments_lengths[i-1]
    # Determine the section to use at each longitudinal position
    s = sections[i]
    # Create the bottom flange points
    s.bf_points.append(create_point_in_modeller(x, +s.bottom_flange_breadth/2, 0.0))
    s.bf_points.append(create_point_in_modeller(x, 0.0,                        0.0))
    s.bf_points.append(create_point_in_modeller(x, -s.bottom_flange_breadth/2, 0.0))
    # Create the top flange points
    s.tf_points.append(create_point_in_modeller(x, +s.top_flange_breadth/2, s.web_depth))
    s.tf_points.append(create_point_in_modeller(x, 0.0,                     s.web_depth))
    s.tf_points.append(create_point_in_modeller(x, -s.top_flange_breadth/2, s.web_depth))

Now join up the segment points. First create helper function to create surfaces and assign a geometric thickness attribute

In [8]:
def create_surface_in_lusas(pnts: list, thk:float, ecc:float, group:str):

    geometryData = lusas.geometryData()
    geometryData.setAllDefaults()
    geometryData.setCreateMethod("coons")
    geometryData.setLowerOrderGeometryType("points")

    obs = lusas.newObjectSet()
    for p in pnts:
        obs.add(p)

    objs = obs.createSurface(geometryData).getObjects("Surface")
    grp = db.getGroupByName(group)
    grp.add(objs, "Surfaces")

    name = f"{group} ({thk:.1f}mm)"

    db.createGeometricSurface(name).setSurface(thk, ecc).assignTo(objs)


Create groups to contain the girder surfaces

In [9]:
db.createEmptyGroup("Top Flange")
db.createEmptyGroup("Web")
db.createEmptyGroup("Bottom Flange")
db.createEmptyGroup("Web Stiffeners")

<COMObject createEmptyGroup>

Here we'll create the surfaces between each section definition using the points we created earlier

In [10]:
for i in range(0, len(segments_lengths)):

    s1:GirderSection = sections[i]   # Section definition at the start of the segment
    s2:GirderSection = sections[i+1] # Section definition at the end of the segment

    create_surface_in_lusas([ s1.bf_points[0], s2.bf_points[0], s2.bf_points[1], s1.bf_points[1] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([ s1.bf_points[1], s2.bf_points[1], s2.bf_points[2], s1.bf_points[2] ], s1.bottom_flange_thk, 0.0, "Bottom Flange")
    create_surface_in_lusas([ s1.bf_points[1], s1.tf_points[1], s2.tf_points[1], s2.bf_points[1] ], s1.web_thk,           0.0, "Web")
    create_surface_in_lusas([ s1.tf_points[0], s2.tf_points[0], s2.tf_points[1], s1.tf_points[1] ], s1.top_flange_thk,    0.0, "Top Flange")
    create_surface_in_lusas([ s1.tf_points[1], s2.tf_points[1], s2.tf_points[2], s1.tf_points[2] ], s1.top_flange_thk,    0.0, "Top Flange")

Create bearing stiffeners at the end sections

In [11]:
for i, thk in enumerate(stiffener_thicknesses):

    if thk > 0:

        s:GirderSection = sections[i]   # Section definition at the start of the segment
        create_surface_in_lusas([ s.bf_points[0], s.tf_points[0], s.tf_points[1], s.bf_points[1] ], thk, 0.0, "Web Stiffeners")
        create_surface_in_lusas([ s.bf_points[1], s.tf_points[1], s.tf_points[2], s.bf_points[2] ], thk, 0.0, "Web Stiffeners")


We'll need to create and assign a shell mesh attribute

In [12]:
# Create the mesh attribute
mesh_attr = db.createMeshSurface("SMsh1")
mesh_attr.setRegularSize("QTS4", mesh_size, True)

# Assign to all 
assignment = lusas.assignment().setAllDefaults().setLoadset(1)
mesh_attr.assignToAll(assignment)

# Mesh the model
db.resetMesh()
db.updateMesh()

<COMObject updateMesh>

And a material attribute

In [13]:
# Create and assign the steel material attribute, we'll do this all inline so we dont hold onto any references
db.createIsotropicMaterial("Steel", 200_000, 0.3, 7.8e-9, 10e-6).assignToAll(lusas.assignment().setAllDefaults().setLoadset(1))

<COMObject assignToAll>

Support attributes

In [14]:
pinned = db.createSupportStructural("Fixed in XYZ").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F")
slide = db.createSupportStructural("Fixed in YZ").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")

# To assign these support attributes we'll need to get hold of the lines. We can do this via the points that have created for each section

# Do this for the start and end sections
for i, section in enumerate([sections[0], sections[-1]]):

    p0 = section.bf_points[0] # point at the +ve flange edge
    p1 = section.bf_points[1] # point at the centre of the web.
    p2 = section.bf_points[2] # point at the -ve flange edge

    # Loop through connected lines 
    for line in p1.getHOFs():
        # Points of the connected line
        ps1 = line.getStartPoint()
        ps2 = line.getEndPoint()
        # Check if the connected line points are those defining the bottom flange
        # If so then we can assign the relevant support attribute
        if ps1 == p0 or ps1 == p2 or ps2 == p0 or ps2 == p2:
            pinned.assignTo(line) if i == 0 else slide.assignTo(line)


Set the model to display the assigned plate thicknesses and supports in an isometric orientation

In [15]:
lusas.view().insertGeometryLayer() # Make sure the drawing layers exist before accessing them
lusas.view().insertAttributesLayer()

lusas.view().geometry().autoColourByAttributes("Geometric", True)
lusas.view().attributes().visualiseAll("Supports")
lusas.view().setIsometric()